# Mesa ABM of the Keep Threshold Relay

### Simulation notes and instructions

Notes:
* Nodes are created simulataneously using the Mesa SimultaneousActivation scheduler
* The nodes go through a bootstrap phase based on Antonio's diagram. They can only form groups if they are in the mainloop forked state.
* Relay requests are currently bernoulli trials
* Each step could be assumed to be a block
* Singature events are modeled by default as asynchronous with a step delay to mimick the block delay

Instructions:
1. Change variables
2. Click on the restart and run kernel icon for ipython

    

In [ ]:
import random
import simpy
import datetime
import numpy as np
import pandas as pd
import agent
import model
import matplotlib.pyplot as plt
import simulation_functions as sc

**Initialize Variables**

In [ ]:
group_size = 10 # Group size
#total_tickets = 500 # total virtual stakers
nodes = 20 # number of nodes
active_group_threshold = 4 # min number of active groups to pick from
signature_threshold = 10  # min number of nodes needed to sign
signature_delay = 2 # lambda for poisson distribution used to generate a delay before the signature is executed
group_expiry = 10 #number of steps after which a group expires
group_formation_threshold = 3 # min number of nodes needed to create a group
node_failure_percent = 10 # % of times nodes fail and have to reconnect
node_death_percent = 5 # % of time nodes fail and do not reconnect


In [ ]:
# Pre-processing Staker distributions
# Linear y = m * (x + 1) since indices start from 0
ticket_distribution = np.zeros(nodes) 

for i in range (0, nodes):
    ticket_distribution[i] =  10 * (i + 1) # m = 50 gives max y as 1000 which is 2% of 50000

x = np.linspace(1,nodes,nodes)
plt.subplot(2, 1, 1)
plt.bar(x, ticket_distribution)
plt.title("Virtual Staker Distribution")

cdf_L02=sc.create_cdf(nodes,ticket_distribution)
plt.subplot(2, 1, 2)
plt.bar(x, cdf_L02)
plt.title("Virtual Staker Cumulative Distribution")

In [ ]:
beacon_model = model.Beacon_Model(nodes, ticket_distribution, 
                                  active_group_threshold, 
                                  group_size, 
                                  signature_threshold, 
                                  group_expiry, 
                                  group_formation_threshold, 
                                  node_failure_percent, 
                                  node_death_percent, 
                                  signature_delay
                                 )

In [ ]:
active_group_size = []
signature_failures = []
for i in range (200):
    beacon_model.step()
    active_group_size.append(len(beacon_model.active_groups))
    print("signature failure events")
    print(beacon_model.unsuccessful_signature_events)
    

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(3, 1, 1)
plt.hist(active_group_size, bins=range(max(active_group_size)+1))
plt.xlabel("# of active groups")
plt.ylabel("frequency")
print(np.median(active_group_size))

plt.subplot(3, 1, 2)
plt.plot(np.linspace(1,len(active_group_size),len(active_group_size)),active_group_size)
plt.xlabel("steps (Blocks)")
plt.ylabel("active groups")

plt.subplot(3, 1, 3)
plt.plot(np.linspace(1,len(beacon_model.unsuccessful_signature_events),
                     len(beacon_model.unsuccessful_signature_events)),
                     beacon_model.unsuccessful_signature_events)
plt.xlabel("Steps (Blocks)")
plt.ylabel("Signature failure events")

In [ ]:

max_group_ownership = []
for group in beacon_model.schedule.agents:
    if group.type == "group":
        #print(group.ownership_distr)
        #print(max(group.ownership_distr)/sum(group.ownership_distr))
        max_group_ownership.append(int(round(max(group.ownership_distr)/sum(group.ownership_distr)*100)))

plt.figure(figsize = (10,10))
plt.subplot(2, 1, 1)
plt.hist(max_group_ownership, bins=range(max(max_group_ownership)+1))
plt.xlabel("Max % group ownership")
plt.ylabel("frequency")
print("Median Group ownership = " + str(np.median(max_group_ownership)))

max_signature_ownership = []
for signature in beacon_model.schedule.agents:
    if signature.type == "signature" and signature.end_signature_process:
        #print(signature.id)
        #print(signature.ownership_distr)
        max_signature_ownership.append(int(round(max(signature.ownership_distr)/sum(signature.ownership_distr)*100)))

plt.subplot(2, 1, 2)
plt.hist(max_signature_ownership, bins=range(max(max_signature_ownership)+1))
plt.xlabel("Max % signature ownership")
plt.ylabel("frequency")
print("Median Signature ownership = " + str(np.median(max_signature_ownership)))

**Further Questions**

* Whats the distribution of group ownership for each node that owns at least 1 group

In [ ]:
# Calculate ownership distributions per node by calculating the node's % ownership of each group
node_ownership_percent= []
for node in beacon_model.schedule.agents:
    if node.type == "node":
        temp = []
        for group in beacon_model.schedule.agents:
            if group.type == "group":
                temp.append(round(group.ownership_distr[node.id]/sum(group.ownership_distr)*100))
        node_ownership_percent.append(temp)

strongest_node = node_ownership_percent.index(max(node_ownership_percent))
weakest_node = node_ownership_percent.index(min(node_ownership_percent))

plt.figure(figsize = (10,10))
plt.subplot(2, 1, 1)
plt.hist(node_ownership_percent[strongest_node], bins=range(int(max(node_ownership_percent[strongest_node]))+1))
plt.xlabel("strongest node group ownership %")
plt.ylabel("frequency")

plt.subplot(2, 1, 2)
plt.hist(node_ownership_percent[weakest_node], bins=range(int(max(node_ownership_percent[weakest_node]))+1))
plt.xlabel("weakest node group ownership %")
plt.ylabel("frequency")


* what % of nodes own at least 1 group

In [ ]:
# count the number of nodes that have no group ownerhship at all, subtract from one to get answer
no_ownership_count = 0
for node in node_ownership_percent:
    if sum(node)==0:
        no_ownership_count +=1
        
print(1-no_ownership_count/nodes)

* How likely is it for a node to have a group that it owns to produce a signature

In [ ]:
# for each group count the number of successful signature events 
group_signature_events = []
group_successes = []
group_sign_ratio = []
for group in beacon_model.schedule.agents:
    if group.type == "group":
        group_sign_total = 0
        group_sign_successes = 0
        #print("group id = "+str(group.id))
        for signature in beacon_model.schedule.agents:
            if signature.type == "signature":
                #print("signature id = "+str(signature.id))
                if signature.group.id == group.id:
                    group_sign_total +=1 # count the total number of signature events this group participates in
                    group_sign_successes += signature.signature_success # count the number of successes
        group_signature_events.append(group_sign_total)
        group_successes.append(group_sign_successes)
        if group_sign_total>0:
            group_sign_ratio.append(round(group_sign_successes/group_sign_total*100))
        else:
            group_sign_ratio.append(0)

print(group_sign_ratio)
print(group_signature_events)
print(group_successes)
            
    
plt.figure(figsize = (10,10))
plt.subplot(2, 1, 1)
plt.hist(group_signature_events, bins=range(int(max(group_signature_events))+1))
plt.xlabel("Group Signature Events")
plt.ylabel("frequency")

plt.subplot(2, 1, 2)
plt.hist(group_sign_ratio, bins=range(int(max(group_sign_ratio))+1))
plt.xlabel("Group sign success ratio")
plt.ylabel("frequency")

